In [1]:
!pip install -q kaggle

In [2]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/

In [3]:
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle datasets download -d tourist55/alzheimers-dataset-4-class-of-images

 88% 30.0M/34.1M [00:00<00:00, 163MB/s]
100% 34.1M/34.1M [00:00<00:00, 163MB/s]


In [5]:
import zipfile
zip_ref = zipfile.ZipFile('/content/alzheimers-dataset-4-class-of-images.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [6]:
!pip install imutils

  Preparing metadata (setup.py) ... done
  Created wheel for imutils: filename=imutils-0.5.4-py3-none-any.whl size=25837 sha256=fb3db378c5cc56d70879e92fae611a8bc46035ce67bbf716d5d8d2fa5dd8297f
  Stored in directory: /root/.cache/pip/wheels/85/cf/3a/e265e975a1e7c7e54eb3692d6aa4e2e7d6a3945d29da46f2d7
Successfully built imutils


In [7]:
import numpy as np
import random
import cv2
import os
from imutils import paths
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from tensorflow.keras.optimizers import legacy

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import backend as K

from fl_implementation_utils import *

# declear path to your mnist data folder
img_path = "/content/Alzheimer_s Dataset/train"

# get the path list using the path object
image_paths = list(paths.list_images(img_path))

# apply our function
image_list, label_list = load(image_paths, input_shape=(100, 100, 3), verbose=10000)

# binarize the labels
lb = LabelBinarizer()
label_list = lb.fit_transform(label_list)

# split data into training and test set
X_train, X_test, y_train, y_test = train_test_split(
    image_list, label_list, test_size=0.1, random_state=42
)

# create clients
clients = create_clients(X_train, y_train, num_clients=10, initial="client")

# process and batch the training data for each client
clients_batched = dict()
for client_name, data in clients.items():
    clients_batched[client_name] = batch_data(data)

# process and batch the test set
test_batched = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(len(y_test))

comms_round = 100

# create optimizer
lr = 0.01
loss = "categorical_crossentropy"
metrics = ["accuracy"]
optimizer = legacy.SGD(lr=lr, decay=lr / comms_round, momentum=0.9)

# initialize global model
smlp_global = PretrainedInceptionModel()
global_model = smlp_global.build(input_shape=(100, 100, 3), num_classes=4)

# commence global training loop
for comm_round in range(comms_round):

    # get the global model's weights - will serve as the initial weights for all local models
    global_weights = global_model.get_weights()

    # initial list to collect local model weights after scalling
    scaled_local_weight_list = list()

    # randomize client data - using keys
    client_names = list(clients_batched.keys())
    random.shuffle(client_names)

    # loop through each client and create new local model
    for client in client_names:
        smlp_local = PretrainedInceptionModel()
        local_model = smlp_local.build(input_shape=(100, 100, 3), num_classes=4)
        local_model.compile(loss=loss, optimizer=optimizer, metrics=metrics)

        # set local model weight to the weight of the global model
        local_model.set_weights(global_weights)

        # fit local model with client's data
        local_model.fit(clients_batched[client], epochs=1, verbose=0)

        # scale the model weights and add to list
        scaling_factor = weight_scalling_factor(clients_batched, client)
        scaled_weights = scale_model_weights(local_model.get_weights(), scaling_factor)
        scaled_local_weight_list.append(scaled_weights)

        # clear session to free memory after each communication round
        K.clear_session()

    # to get the average over all the local model, we simply take the sum of the scaled weights
    average_weights = sum_scaled_weights(scaled_local_weight_list)

    # update global model
    global_model.set_weights(average_weights)

    # test global model and print out metrics after each communications round
    for X_test, Y_test in test_batched:
        global_acc, global_loss = test_model(X_test, Y_test, global_model, comm_round)
SGD_dataset = (
    tf.data.Dataset.from_tensor_slices((X_train, y_train))
    .shuffle(len(y_train))
    .batch(320)
)
smlp_SGD = PretrainedInceptionModel()
SGD_model = smlp_SGD.build(input_shape=(100, 100, 3), num_classes=4)

SGD_model.compile(loss=loss, optimizer=optimizer, metrics=metrics)

# fit the SGD training data to model
_ = SGD_model.fit(SGD_dataset, epochs=100, verbose=0)

# test the SGD global model and print out metrics
for X_test, Y_test in test_batched:
    SGD_acc, SGD_loss = test_model(X_test, Y_test, SGD_model, 1)


/usr/local/lib/python3.10/dist-packages/keras/src/optimizers/legacy/gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


17/17 [==============================] - 2s 54ms/step
comm_round: 0 | global_acc: 49.318% | global_loss: 1.3186146020889282
17/17 [==============================] - 1s 54ms/step
comm_round: 1 | global_acc: 49.318% | global_loss: 1.2622722387313843
17/17 [==============================] - 1s 54ms/step
comm_round: 2 | global_acc: 51.072% | global_loss: 1.236676573753357
17/17 [==============================] - 1s 55ms/step
comm_round: 3 | global_acc: 53.996% | global_loss: 1.223749041557312
17/17 [==============================] - 1s 54ms/step
comm_round: 4 | global_acc: 52.827% | global_loss: 1.217298150062561
17/17 [==============================] - 1s 54ms/step
comm_round: 5 | global_acc: 54.971% | global_loss: 1.2275115251541138
17/17 [==============================] - 2s 54ms/step
comm_round: 6 | global_acc: 55.556% | global_loss: 1.2318998575210571
17/17 [==============================] - 2s 57ms/step
comm_round: 7 | global_acc: 55.361% | global_loss: 1.2267135381698608
17/17 [====